In [1]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.classification import LogisticRegression 
#training data with label and features 
training = spark.createDataFrame([ 
 (1.0, Vectors.dense([0.0, 1.1, 0.1])), 
 (0.0, Vectors.dense([2.0, 1.0, -1.0])), 
 (0.0, Vectors.dense([2.0, 1.3, 1.0])), 
 (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"]) 


In [2]:
lr = LogisticRegression(maxIter=10, regParam=0.01) 
Model1=lr.fit(training) 


2024-09-24 11:17:09,636 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2024-09-24 11:17:09,637 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [3]:
training.show(10,False) 


+-----+--------------+
|label|features      |
+-----+--------------+
|1.0  |[0.0,1.1,0.1] |
|0.0  |[2.0,1.0,-1.0]|
|0.0  |[2.0,1.3,1.0] |
|1.0  |[0.0,1.2,-0.5]|
+-----+--------------+



In [4]:
paramMap = {lr.maxIter: 20} 
paramMap[lr.maxIter] = 30 # Specify 1 Param, overwriting the original maxIter. 
# Specify multiple Params. 
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55}) 
paramMap2 = {lr.probabilityCol: "myProbability"} # type: ignore 
paramMapCombined = paramMap.copy() 
paramMapCombined.update(paramMap2) # type: ignore 
# paramMapCombined overrides all parameters set earlier via lr.set* methods. 
model2 = lr.fit(training, paramMapCombined)

In [7]:
 test = spark.createDataFrame([ 
 (1.0, Vectors.dense([-1.0, 1.5, 1.3])), 
 (0.0, Vectors.dense([3.0, 2.0, -0.1])), 
 (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"]) 
# Make predictions on test data using the Transformer.transform() method. 
# LogisticRegression.transform will only use the 'features' column. 
# Note that model2.transform() outputs a "myProbability" column instead of the usual 
# 'probability' column since we renamed the lr.probabilityCol parameter previously. 
prediction = model2.transform(test) 
result = prediction.select("features", "label", "myProbability", "prediction").collect() 
for row in result: 
    print("features=%s, label=%s -> prob=%s, prediction=%s" 
 % (row.features, row.label, row.myProbability, row.prediction)) 


features=[-1.0,1.5,1.3], label=1.0 -> prob=[0.0570730417103402,0.9429269582896598], prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> prob=[0.9238522311704105,0.07614776882958951], prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> prob=[0.10972776114779453,0.8902722388522055], prediction=1.0
